In [1]:
!pip install wandb --quiet
!pip install swig --quiet
!pip install gym[all] --quiet
!pip install pygame --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.7 MB/s eta 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
## fixing the pygame error

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

pygame 2.1.0 (SDL 2.0.16, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [4]:
import gym
from gym.envs import box2d
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [5]:
class Policy_NN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size = 128, gamma = 0.99, learning_rate = 0.001):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

        self.optimizer = optim.Adam(self.parameters(),lr=self.learning_rate)


    def forward(self,x):
      x = self.layer1(x) #forward propagation
      x = self.layer2(x)
      x = self.softmax(x)
      return x

    def act(self,state):
        #With obs, find the maximum Q value and output the corresponding action
        #action_probs = torch.softmax(self.actor(obs), dim=-1)
        #state_value = self.critic(state)

        #obs_tensor = torch.as_tensor(obs, dtype=torch.float32)

        #action = log_value.detach().item() #action corresponding to max Q index
        #return action, log_value
        #print("checkpoint 6")
        action_probs = torch.softmax(self.policy_network(state), dim=-1)
        # q_value = self(obs_tensor.unsqueeze(0)) #convert to row vector
        # highest_q_value = torch.argmax(input=q_value)
        # log_value = torch.log(q_value.squeeze(0)[highest_q_value])
        #print("checkpoint 7")
        #print("action_probs: {}".format(action_probs))
        #state_value = self.critic(state)
        return action_probs

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
class update_function:
    def __init__(self, Policy_NN, rewards, log_probs_array):
        self.network = Policy_NN
        self.rewards = rewards
        self.log_value = log_probs_array

    def update_function(self):
        discount_rewards = []
        Gt,pw = 0, 0
        for r in reversed(self.rewards):
            Gt = Gt + gamma**pw * r
            pw = pw + 1
            discount_rewards.append(Gt)

        steps_value = np.arange(len(self.rewards))

        discount_rewards = torch.tensor(discount_rewards)
        discount_rewards = discount_rewards[::-1].cumsum()[::-1] / gamma**steps_value

        policy_gradient = []
        for log_prob, e_d_r in zip(self.log_value, discount_rewards):
            policy_gradient.append(-log_prob*e_d_r)

        policy_network.optimizer.zero_grad()
        policy_gradient = torch.stack(policy_gradient).sum()
        policy_gradient.backward()
        policy_network.optimizer.step()

        return policy_gradient

In [7]:
def update(policy_network, gamma = 0.99):
    log_probs = []
    rewards = []

In [8]:
import torch
import numpy as np
import gym
import statistics

In [19]:
def run(policy_network, env, n_episode):
    episodic_reward = []
    for episode in range(n_episode):
        state = env.reset()
        done = False
        log_probs_array = []
        total_reward = 0
        while not done:
            stateT = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            action_probs = policy_network(stateT)
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action.unsqueeze(0))
            log_probs_array.append(log_prob)
            next_state, reward, done, _ = env.step(action.item())
            total_reward += reward
            update_function(policy_network, rewards,log_probs_array)
            state = next_state

        print("Episode:{} | Reward: {}".format(episode, total_reward))
        episodic_reward.append(total_reward)
        ennv.close()
    return episodic_reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
def main():
    env = gym.make("LunarLander-v2")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    policy_network = Policy_NN(state_size, action_size)

    n_episode = 500
    n_timesteps = 1000
    episode_array = [i for i in range(1,n_episode+1)]
    rewards_array = run(policy_network,env,n_episode)
    # discounted_rewards = []
    # R = 0
    # for reward in rewards:
    #     R = reward + policy_network.gamma * R
    #     discounted_rewards.insert(0,R)

    # # Compute policy loss
    # policy_loss = torch.stack(log_probs_array) * discounted_rewards
    # policy_loss = -policy_loss.sum()

    # # Update policy
    # optimizer.zero_grad()
    # policy_loss.backward()
    # optimizer.step()

In [16]:
main()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Streaming output truncated to the last 5000 lines.
Episode:495 |  Step: 0 | action: 0 | Reward: -2.147518321419824
Episode:495 |  Step: 1 | action: 0 | Reward: -2.089382808134701
Episode:495 |  Step: 2 | action: 1 | Reward: -1.9056820905832967
Episode:495 |  Step: 3 | action: 0 | Reward: -1.8179658295998138
Episode:495 |  Step: 4 | action: 3 | Reward: -1.973710882666693
Episode:495 |  Step: 5 | action: 2 | Reward: 1.9438015922698184
Episode:495 |  Step: 6 | action: 3 | Reward: -2.0870566339969217
Episode:495 |  Step: 7 | action: 1 | Reward: -1.866587445072837
Episode:495 |  Step: 8 | action: 2 | Reward: 3.183512211066227
Episode:495 |  Step: 9 | action: 0 | Reward: -1.9062037836728791
Episode:495 |  Step: 10 | action: 3 | Reward: -1.9456451769298428
Episode:495 |  Step: 11 | action: 0 | Reward: -1.9471803002792285
Episode:495 |  Step: 12 | action: 3 | Reward: -2.1264184444497958
Episode:495 |  Step: 13 | action: 0 | Reward: -2.0460789378378763
Episode:495 |  Step: 14 | action: 3 | Rewa

NameError: name 'log_probs' is not defined

In [18]:
plt.title("Performance of REINFORCE")
plt.plot(episode_array, reward_array, label = "REINFORCE")
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.legend()
plt.show()

NameError: name 'episode_array' is not defined